# Creating custom crawlers with `advertools` - with one line of code

Many times you run repeated crawls with the same settings and options, depending on the type of crawl that tyou want to run.

This is a way to utilize Python's `partial` function from the `functools` module to achieve that with a single line of code.

TL; DR

Let's make an example:

In [1]:
from functools import partial
import advertools as adv

Let's say you want to create a crawler that runs in spider mode, and stops after having crawled X requests. This is what you typically do the first time, when you are just exploring a new website.

Let's call it `exploratory_crawler`.

The `partial` function allows us to take an existing function, and set default values to its parameters. The default in `adv.crawl` is that `follow_links=False`. So we want this to default to `True`.

Here's how we do it:

In [2]:
exploratory_crawler = partial(adv.crawl, follow_links=True)

That's it!

Now we have a new function `exploratory_crawler` (which is essentially `adv.crawl` with new defaults) that we can use normally:

In [5]:
exploratory_crawler(url_list='https://adver.tools', output_file='test_crawl.jl') # this will run with follow_links=True

In [6]:
import pandas as pd
crawldf = pd.read_json('test_crawl.jl', lines=True)
crawldf

,url,title,meta_desc,viewport,h1,h4,body_text,size,download_timeout,download_slot,...,request_headers_Accept,request_headers_Accept-Language,request_headers_User-Agent,request_headers_Accept-Encoding,redirect_times,redirect_ttl,redirect_urls,redirect_reasons,request_headers_Referer,h2
0,https://adver.tools,advertools,Get productive and get insights for your digit...,"width=device-width, initial-scale=1.0, maximum...",advertools: online marketing productivity and ...,@@@@@@@@@@@@@@@@@@,\n \n \n \...,5270,180,adver.tools,...,"text/html,application/xhtml+xml,application/xm...",en,advertools/0.14.2,"gzip, deflate, br",NaN,NaN,NaN,NaN,NaN,NaN
1,https://adver.tools/audience-manager/,"Audience Manager - Compare, Mix, & Match Audie...",NaN,"width=device-width, initial-scale=1.0, maximum...","Compare, Merge, and Analyze Audience Lists",NaN,\n \n \n \...,8897,180,adver.tools,...,"text/html,application/xhtml+xml,application/xm...",en,advertools/0.14.2,"gzip, deflate, br",1.0,19.0,https://adver.tools/audience-manager,308.0,https://adver.tools,NaN
2,https://adver.tools/link-analysis/,Analyze internal and external links – advertools,NaN,"width=device-width, initial-scale=1.0, maximum...",Internal Link Analysis Tool,NaN,\n \n \n \...,10642,180,adver.tools,...,"text/html,application/xhtml+xml,application/xm...",en,advertools/0.14.2,"gzip, deflate, br",1.0,19.0,https://adver.tools/link-analysis,308.0,https://adver.tools,What is internal link analysis?@@How to analyz...
3,https://adver.tools/user-agent-parser/,User-agent parser - advertools,NaN,"width=device-width, initial-scale=1.0, maximum...",Bulk User-agent Parser,NaN,\n \n \n \...,8845,180,adver.tools,...,"text/html,application/xhtml+xml,application/xm...",en,advertools/0.14.2,"gzip, deflate, br",1.0,19.0,https://adver.tools/user-agent-parser,308.0,https://adver.tools,NaN
4,https://adver.tools/xml-sitemaps/,Analyze XML Sitemaps – advertools,Download XML sitemaps using normal sitemap URL...,"width=device-width, initial-scale=1.0, maximum...","Download, parse, and analyze XML sitemaps",NaN,\n \n \n \...,14996,180,adver.tools,...,"text/html,application/xhtml+xml,application/xm...",en,advertools/0.14.2,"gzip, deflate, br",1.0,19.0,https://adver.tools/xml-sitemaps,308.0,https://adver.tools,NaN
5,https://adver.tools/content-similarity/,Content Similarity - advertools,NaN,"width=device-width, initial-scale=1.0, maximum...",Content Similarity,NaN,\n \n \n \...,8854,180,adver.tools,...,"text/html,application/xhtml+xml,application/xm...",en,advertools/0.14.2,"gzip, deflate, br",1.0,19.0,https://adver.tools/content-similarity,308.0,https://adver.tools,NaN
6,https://adver.tools/urlytics/,Split and Analyze URL Structure – advertools,Analyze URL structure. This tool splits a list...,"width=device-width, initial-scale=1.0, maximum...",Analyze URL Structure,NaN,\n \n \n \...,8978,180,adver.tools,...,"text/html,application/xhtml+xml,application/xm...",en,advertools/0.14.2,"gzip, deflate, br",1.0,19.0,https://adver.tools/urlytics,308.0,https://adver.tools,NaN
7,https://adver.tools/entity-extraction/,Entity Extraction powered by OpenAI's ChatGPT ...,NaN,"width=device-width, initial-scale=1.0, maximum...",Entity Extraction powered by OpenAI's ChatGPT,Example input:@@Example output,\n \n \n \...,11140,180,adver.tools,...,"text/html,application/xhtml+xml,application/xm...",en,advertools/0.14.2,"gzip, deflate, br",1.0,19.0,https://adver.tools/entity-extraction,308.0,https://adver.tools,What is entity extraction?@@The value of entit...
8,https://adver.tools/reverse-dns-lookup/,Bulk reverse DNS lookup - advertools,NaN,"width=device-width, initial-scale=1.0, maximum...",Bulk Reverse DNS Lookup,NaN,\n \n \n \...,8864,180,adver.tools,...,"text/html,application/xhtml+xml,application/xm...",en,advertools/0.14.2,"gzip, deflate, br",1.0,19.0,https://adver.tools/reverse-dns-lookup,308.0,https://adver.tools,NaN
9,https://adver.tools/seo-crawler/,Website and SEO Crawler - advertools,NaN,"width=device-width, initial-scale=1.

Done!

Let's also set a default number of pages, after which we want the crawler to stop, because we are just exploring and don't want to wait for 50k pages to be crawled.

We use the same approach with an additional default option (feel free to change the value for `CLOSESPIDER_PAGECOUNT` to anything else):

In [14]:
exploratory_crawler = partial(
    adv.crawl,
    follow_links=True,
    custom_settings={
        'CLOSESPIDER_PAGECOUNT': 2000
    })

Now `exploratory_crawler` would follow links and stop after having crawled 2k pages.

## More customization

Let's say we want to keep the defaults as is, but we want to add a new option.

If we override `custom_settings` we would overwrite all the previous defaults, which we don't want. We simply want to update them.

We can explor the `partial` object by checking three attributes:

In [16]:
exploratory_crawler.func # this is the original function we are overriding

<function advertools.spider.crawl(url_list, output_file, follow_links=False, allowed_domains=None, exclude_url_params=None, include_url_params=None, exclude_url_regex=None, include_url_regex=None, css_selectors=None, xpath_selectors=None, custom_settings=None)>

In [17]:
exploratory_crawler.args # we didn't use positional arguments, so it's empty

()

In [18]:
exploratory_crawler.keywords # we used some keyword args, which we want to update

{'follow_links': True, 'custom_settings': {'CLOSESPIDER_PAGECOUNT': 2000}}

Let's say we want to write the crawl logs to a certain log file:

In [22]:
exploratory_crawler.keywords['custom_settings'].update({'LOG_FILE': 'path/to/your/logfile.log'})

Let's see if it was updated:

In [23]:
exploratory_crawler

functools.partial(<function crawl at 0x1200fcc20>, follow_links=True, custom_settings={'CLOSESPIDER_PAGECOUNT': 2000, 'LOG_FILE': 'path/to/your/logfile.log'})

There you go.

Here is a bunch of custom crawlers that you can create with a single line of code each:

## User-agent and device-based crawling

In order to customize further, it would be good to have a set of the most popular User-agents, and have them classified according to devicy type, operating system, version, etc.

So let's get some user-agents (this can be improved with a more comprehensive list but it should provide enough variety)

In [85]:
# from ua_parser import user_agent_parser
# import requests
# resp = requests.get('https://gist.githubusercontent.com/pzb/b4b6f57144aea7827ae4/raw/cf847b76a142955b1410c8bcef3aabe221a63db1/user-agents.txt')
# resp.text.splitlines()

# ua_df = pd.json_normalize([user_agent_parser.Parse(ua) for ua in resp.text.splitlines()], sep='_')
# ua_df.to_csv('ua_list.csv', index=False)

In [98]:
ua_df = pd.read_csv('ua_list.csv')
ua_df.sample(10)

,string,user_agent_family,user_agent_major,user_agent_minor,user_agent_patch,os_family,os_major,os_minor,os_patch,os_patch_minor,device_family,device_brand,device_model
863,Mozilla/5.0 (Windows NT 6.1; rv:29.0) Gecko/20...,Firefox,29.0,0.0,NaN,Windows,7,NaN,NaN,NaN,Other,NaN,NaN
800,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2)...,Chrome,45.0,0.0,2454.0,Mac OS X,10,8.0,2.0,NaN,Mac,Apple,Mac
543,Mozilla/5.0 (Linux; Android 5.1.1; Nexus 10 Bu...,Chrome,45.0,0.0,2454.0,Android,5,1.0,1.0,NaN,Asus Nexus 10,Asus,Nexus 10
713,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...,IE,8.0,0.0,NaN,Windows,XP,NaN,NaN,NaN,Other,NaN,NaN
725,Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKi...,Chrome,39.0,0.0,2171.0,Windows,8,NaN,NaN,NaN,Other,NaN,NaN
266,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,Chrome,44.0,0.0,2403.0,Windows,7,NaN,NaN,NaN,Other,NaN,NaN
278,Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7....,IE,11.0,0.0,NaN,Windows,8,1.0,NaN,NaN,Other,NaN,NaN
839,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,Chrome,31.0,0.0,1650.0,Windows,7,NaN,NaN,NaN,Other,NaN,NaN
820,Mozilla/5.0 (Linux; Android 5.0.1; SAMSUNG SM-...,Samsung Internet,2.0,1.0,NaN,Android,5,0.0,1.0,NaN,Samsung SM-N910T,Samsung,SM-N910T
286,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...,IE,8.0,0.0,NaN,Windows,XP,NaN,NaN,NaN,Other,NaN,NaN


## Exploratory crawler: as explained above

In [ ]:
exploratory_crawler = partial(
    adv.crawl,
    follow_links=True,
    custom_settings={
        'CLOSESPIDER_PAGECOUNT': 2000
    })

## Rude crawler:
* Spider mode: on
* Does not respect robots.txt rules

In [ ]:
rude_crawler = partial(
    adv.crawl,
    follow_links=True,
    custom_settings={
        'ROBOTSTXT_OBEY': False
    })

## Polite crawler
* Respects robots.txt rules (it's the default anyway)
* Autothrottling enabled (changes crawling speed dynamically)
* Targets a concurrency of 1. Runs one request at a time
* Waits five seconds between crawled pages

In [ ]:
polite_crawler = partial(
    adv.crawl,
    follow_links=True,
    custom_settings={
        'AUTOTHROTTLE_ENABLED': True,
        'AUTOTHROTTLE_TARGET_CONCURRENCY': 1,
        'DOWNLOAD_DELAY': 5
    })

## Greenlight crawler
* Run with 48 concurrent requests per domain
* Assumes you have authority to do so, otherwise you'll probably get blocked
* Don't obey robots.txt rules, see what you end up crawling

In [ ]:
# I wanted to call it DDOS crawler, but it sounded evil
greenlight_crawler = partial(
    adv.crawl,
    follow_links=True,
    custom_settings={
        'ROBOTSTXT_OBEY': False,
        'CONCURRENT_REQUESTS_PER_DOMAIN': 48,
    })

## My User-agent crawler
* Spider mode on
* Runs with a custom user agent (in cases where normal robots would be blocked and you have permission to crawl)

In [ ]:
my_ua_crawler = partial(
    adv.crawl,
    follow_links=True,
    custom_settings={
        'USER_AGENT': 'MY CUSTOM USER-AGENT'
    })

## Shallow crawler
* Crawl, follow links, and stop after 2 links from the start URLs (feel free to change 2 to any other number)

In [ ]:
shallow_crawler = partial(
    adv.crawl,
    follow_links=True,
    custom_settings={
        'DEPTH_LIMIT': 2
    })

## No-params crawler
* Spider mode: on
* Follow links, but only links that **don't** have URL query parameters

In [ ]:
no_params_crawler = partial(
    adv.crawl,
    follow_links=True,
    exclude_url_params=True
)

## Incremental crawler
* Rerun the same crawl every month/week/day/hour
* Don't crawl pages that have already been crawled
* Every time, stop after X pages have been crawled
* Save crawl logs to a file, so you can check (this is a good practice to always use)

In [ ]:
incremental_crawler = partial(
    adv.crawl,
    follow_links=True,
    custom_settings={
        'JOBDIR': 'path/to/your/jobdir',  # <-- change this for every crawling job (website)
        'CLOSESPIDER_PAGECOUNT': 500,
        'LOG_FILE': 'path/to/your/log_file.log'
    })

## Random User-agent crawler

* Spider mode: on
* Uses a random user agent from a provided list os UA's
* You need to install a separate package with `pip install scrapy-random-useragent`
* You also need to create a text file with your selected user agents in it, one per line
* Use the `string` column from `ua_df` for getting a random list of UA's and save them to a text file:

In [86]:
ua_df['string']

0      Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
1      Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
2      Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....
3      Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...
4      Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5...
                             ...                        
995    Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...
996    Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5...
997    Mozilla/5.0 (Windows NT 6.3; Win64; x64; Tride...
998    Mozilla/5.0 (Linux; Android 5.0; SAMSUNG SM-N9...
999    Mozilla/5.0 (iPhone; CPU iPhone OS 8_4 like Ma...
Name: string, Length: 1000, dtype: object

In [ ]:

user_agent_file_path = '/path/to/list/of/useragents.txt'

rand_ua_crawler = partial(
    adv.crawl,
    follow_links=True,
    custom_settings={
    'DOWNLOADER_MIDDLEWARES': {
        'scrapy.contrib.downloadermiddleware.useragent.UserAgentMiddleware': None,
        'random_useragent.RandomUserAgentMiddleware': 400
    },
    'USER_AGENT_LIST': user_agent_file_path,
    })

Here are some more examples based on user-agent, and feel free to mix and match depending on your needs

## Android user-agent crawler

In [87]:
android_ua = ua_df[ua_df['os_family'].eq('Android')]
android_ua.head()

,string,user_agent_family,user_agent_major,user_agent_minor,user_agent_patch,os_family,os_major,os_minor,os_patch,os_patch_minor,device_family,device_brand,device_model
51,Mozilla/5.0 (Linux; U; Android 4.0.3; en-us; K...,Amazon Silk,3.0,68.0,NaN,Android,4,0.0,3.0,NaN,Kindle Fire HD,Amazon,"Kindle Fire HD 7"""
55,Mozilla/5.0 (Linux; U; Android 4.4.3; en-us; K...,Amazon Silk,3.0,68.0,NaN,Android,4,4.0,3.0,NaN,"Kindle Fire HDX 7"" WiFi",Amazon,"Kindle Fire HDX 7"" WiFi"
77,Mozilla/5.0 (Linux; U; Android 4.4.3; en-us; K...,Amazon Silk,3.0,68.0,NaN,Android,4,4.0,3.0,NaN,Kindle,Amazon,Kindle
87,Mozilla/5.0 (Linux; U; Android 4.0.4; en-us; K...,Amazon Silk,3.0,68.0,NaN,Android,4,0.0,4.0,NaN,"Kindle Fire HD 8.9"" WiFi",Amazon,"Kindle Fire HD 8.9"" WiFi"
101,Mozilla/5.0 (Linux; U; Android 4.4.3; en-us; K...,Amazon Silk,3.0,68.0,NaN,Android,4,4.0,3.0,NaN,"Kindle Fire HD 7"" WiFi",Amazon,"Kindle Fire HD 7"" WiFi"


In [89]:
android_ua['string']  # save these to a text file

51     Mozilla/5.0 (Linux; U; Android 4.0.3; en-us; K...
55     Mozilla/5.0 (Linux; U; Android 4.4.3; en-us; K...
77     Mozilla/5.0 (Linux; U; Android 4.4.3; en-us; K...
87     Mozilla/5.0 (Linux; U; Android 4.0.4; en-us; K...
101    Mozilla/5.0 (Linux; U; Android 4.4.3; en-us; K...
                             ...                        
947    Mozilla/5.0 (Linux; Android 4.2.2; Le Pan TC80...
974    Mozilla/5.0 (Linux; Android 4.4.4; Nexus 7 Bui...
975    Mozilla/5.0 (Linux; Android 4.2.2; QMV7B Build...
978    Mozilla/5.0 (Linux; U; Android 4.0.3; en-us) A...
998    Mozilla/5.0 (Linux; Android 5.0; SAMSUNG SM-N9...
Name: string, Length: 114, dtype: object

In [ ]:
android_ua_file_path = '/path/to/list/of/android_user_agents.txt'

android_ua_crawler = partial(
    adv.crawl,
    follow_links=True,
    custom_settings={
    'DOWNLOADER_MIDDLEWARES': {
        'scrapy.contrib.downloadermiddleware.useragent.UserAgentMiddleware': None,
        'random_useragent.RandomUserAgentMiddleware': 400
    },
    'USER_AGENT_LIST': android_ua_file_path,
    })

## iOS crawler

In [95]:
ios_ua = ua_df['os_family'].eq('iOS')  # save these to a text file

In [96]:
ios_ua_file_path = '/path/to/list/of/ios_user_agents.txt'

ios_ua_crawler = partial(
    adv.crawl,
    follow_links=True,
    custom_settings={
    'DOWNLOADER_MIDDLEWARES': {
        'scrapy.contrib.downloadermiddleware.useragent.UserAgentMiddleware': None,
        'random_useragent.RandomUserAgentMiddleware': 400
    },
    'USER_AGENT_LIST': ios_ua_file_path,
    })

## Sitemap crawler
* A simple actually
* Provide a URL for a sitemap (normal, sitemapindex, or a robots.txt URL)
* It crawls those URLs only (list mode)

In [ ]:
def sitemap_crawler(sitemap, output_file):
    sitemap_df = adv.sitemap_to_df(sitemap)
    adv.crawl(url_list=sitemap_df['loc'], output_file=output_file)

## Random crawler
* Introduce lots of randomness, might uncover some insights one one of the parameters (country, user agent, etc.)
* Don't obey robots.txt rules
* Randomize download delays
* Rancomize user-agents
* Additionally, check out [how to run with random proxies](https://advertools.readthedocs.io/en/master/advertools.code_recipes.spider_strategies.html#how-do-i-use-a-proxy-while-crawling) here for more randomness

In [97]:
user_agent_file_path = '/path/to/list/of/useragents.txt'

random_crawler = partial(
    adv.crawl,
    follow_links=True,
    custom_setting={
        'ROBOTSTXT_OBEY': False,
        'DOWNLOADER_MIDDLEWARES': {
            'scrapy.contrib.downloadermiddleware.useragent.UserAgentMiddleware': None,
            'random_useragent.RandomUserAgentMiddleware': 400
        },
        'USER_AGENT_LIST': user_agent_file_path,
        'DOWNLOAD_DELAY': 3,
        'RANDOMIZE_DOWNLOAD_DELAY': True
    })

## Crawl If-Modified-Since

* Crawl a list of URLs only if they were modified since a given date
* Dates need to be formatted with the format string `%a, %d %b %Y %H:%M:%S %Z`
* Example date: `Thu, 25 Apr 2024 15:09:30 UTC`
* You can have a separate modified date per URL

In [ ]:
date_format = '%a, %d %b %Y %H:%M:%S %Z'

def crawl_if_modified_since(url_list, lastmod_list):
    """Crawl the `url_list` only if they were modified before `lastmod_list`.

    You can have a separate date per URL.

    Parameters
    ----------
    url_list : list
        A list of URLs to crawl.
    lastmod_list : list
        A list of dates to check agains. Date format: "%a, %d %b %Y %H:%M:%S %Z"
    """
    for url, lastmod in zip(url_list, lastmod_list):
        adv.crawl(
            url_list=url,
            output_file='output_file.jl',
            custom_settings={'DEFAULT_REQUEST_HEADERS': {'If-Modified-Since': lastmod}})